In [1]:
import tensorflow as tf
import pandas as pd
from Function import *
import numpy as np

In [2]:
#입력 Data가 잘못되어 Nan or Inf 발생할 수 있음. 그때를 대비하여 정규화 코드 첨부.
def MinMaxScaler(data):
    return data/45
def N(data):
    return data*45

def to_float(data):
    data = data/45
    return 45*data

In [4]:
Data = read_csv('CSV_Data2.csv')
Data = MinMaxScaler(Data)
x_data = [[[0 for i in range(45)] for j in range(7)] for k in range(796)]
y_data = pd.DataFrame(data=0, columns=range(0,796), index=range(0,7))

#Data

In [5]:
#lotto data is loaded to x_data, y_data
#y_data has pulled 1 column to left. because y_data would be compared with x_data. 
#y_data's last column is NaN to predict 
#x_data = Data
for col in range(796):
    for row in range(7):
        #print("row : ", row)
        for data in range(0, 45):
            if(float(data) == Data[col][row]):
                x_data[col][row][data] = 1

for col in range(795):#열
    for row in range(7):#행
        y_data[col][row] = Data[col+1][row]
        
y_data = MinMaxScaler(y_data)
#y_data = N(y_data)
#y_data = to_float(y_data)

In [6]:
#parameter
batch_size = 1
sequence_length = 796
input_dimention = 7
output_dimention = 7
hidden_size = 10

In [7]:
tf.reset_default_graph()
X = tf.placeholder(dtype=tf.float32, shape=[batch_size, input_dimention, 45],name="X")#(1,7,45)
Y = tf.placeholder(dtype=tf.int32, shape=[batch_size, output_dimention])#(1,7)

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
initial_state = cell.zero_state(batch_size=batch_size, dtype=tf.float32)

#initial_state = tf.Variable(cell.zero_states(batch_size, tf.float32), trainable=False)
output, state = tf.nn.dynamic_rnn(cell=cell, inputs=X, dtype=tf.float32, initial_state=initial_state)

W = tf.Variable(tf.random_normal(dtype=tf.float32, shape=[batch_size, 7]))

In [8]:
#Model & Fully connected layer
#현재 output -> [1,7,10]
output_fclayer_buff = tf.reshape(tensor=output, shape=[-1, hidden_size])
#output_fclayer_buff = [7, 10]

output = tf.contrib.layers.fully_connected(inputs=output_fclayer_buff, num_outputs=45, activation_fn=None)
#output.shape => [7, 7] Hidden_Layer가 무엇이든 7,7로 나옴.
print(output.shape)
output = tf.reshape(tensor=output, shape=[1, 7, 45])

A = tf.contrib.seq2seq.sequence_loss(logits=output, targets=Y, weights=W)
cost = tf.reduce_mean(A)
optimizer = tf.train.AdamOptimizer(learning_rate=0.05).minimize(cost)

(7, 45)


In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(10):
        for i in range(795):
            cost_val, _, _A = sess.run([cost, optimizer, output], feed_dict= {X : [x_data[i]], Y : [y_data[i]]})
        print("epoch : ", epoch+1, "cost : ", cost_val)
        print(_A)
    

epoch :  1 cost :  0.001194375
[[[  4.558319   -4.279672   -4.146049   -4.1456947  -4.227423
    -4.076259   -4.1991286  -4.0888286  -4.0458217  -4.098567
    -4.042492   -3.792922   -3.9350019  -4.1293354  -3.98833
    -4.358917   -4.1396036  -4.036661   -4.1778073  -4.197664
    -4.196313   -4.1850796  -3.7657707  -4.052911   -3.7763073
    -3.882395   -4.042543   -4.184836   -4.289185   -4.302516
    -4.189719   -3.9469652  -4.130038   -4.1748056  -3.9896255
    -4.0579515  -4.223536   -4.489219   -4.393653   -4.1252723
    -4.3530645  -3.9426484  -4.2073555  -4.356562   -4.598057 ]
  [  9.762539   -9.057276   -8.730923   -8.72174    -8.932127
    -8.543229   -8.862927   -8.584205   -8.4595175  -8.612169
    -8.452732   -7.832089   -8.194829   -8.672155   -8.317594
    -9.271074   -8.699806   -8.434642   -8.8053055  -8.843016
    -8.859929   -8.822025   -7.7572565  -8.48497    -7.788663
    -8.056896   -8.465495   -8.821007   -9.100151   -9.121845
    -8.834524   -8.222959   -8.6973